<a href="https://colab.research.google.com/github/ThusithaDeepal/predictionmodel/blob/master/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras import Sequential
from tensorflow.keras.activations import sigmoid

In [26]:
import pandas as pd
df = pd.read_csv('kaggle_data_1h.csv')
df = df[['time', 'Voltage']]  # Fix the KeyError by using double brackets [[]] to select multiple columns
df['time'] = pd.to_datetime(df['time'])

# Set the time column as the index
df = df.set_index('time')
df

,Voltage
time,
2006-12-16 17:00:00,234.643889
2006-12-16 18:00:00,234.580167
2006-12-16 19:00:00,233.232500
2006-12-16 20:00:00,234.071500
2006-12-16 21:00:00,237.158667
...,...
2010-11-26 17:00:00,237.069667
2010-11-26 18:00:00,237.531833
2010-11-26 19:00:00,236.741000


In [33]:
x_train, x_test, y_train, y_test = train_test_split(df.index, df['Voltage'], test_size=0.2, random_state=0, shuffle=False)

In [45]:
model=Sequential()
model.add(Dense(12,input_dim=5,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1,activation='linear'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 12)                72        
                                                                 
 dense_10 (Dense)            (None, 8)                 104       
                                                                 
 dense_11 (Dense)            (None, 1)                 9         
                                                                 
Total params: 185 (740.00 Byte)
Trainable params: 185 (740.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [46]:
model.compile(loss='mse',optimizer='adam',metrics=['mse','mae'])

In [47]:
# Use the TimeSeriesGenerator class to prepare the data
train = df[:int(0.8*(len(df)))]
test = df[int(0.8*(len(df))):]
seq_len = 5
from keras.preprocessing.sequence import TimeseriesGenerator
train_generator = TimeseriesGenerator(train['Voltage'].values, train['Voltage'].values, length=seq_len, batch_size=32)
test_generator = TimeseriesGenerator(test['Voltage'].values, test['Voltage'].values, length=seq_len, batch_size=32)

In [48]:
# Train the model  to predict the voltage
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, min_delta=0.0001)
history = model.fit(train_generator, epochs=50, validation_data=test_generator)

Epoch 1/50
855/855 [==============================] - 3s 2ms/step - loss: 40.2600 - mse: 40.2600 - mae: 2.7665 - val_loss: 4.1292 - val_mse: 4.1292 - val_mae: 1.5665
Epoch 2/50
855/855 [==============================] - 2s 2ms/step - loss: 4.9735 - mse: 4.9735 - mae: 1.7237 - val_loss: 3.3920 - val_mse: 3.3920 - val_mae: 1.4146
Epoch 3/50
855/855 [==============================] - 2s 2ms/step - loss: 4.1052 - mse: 4.1052 - mae: 1.5526 - val_loss: 2.9319 - val_mse: 2.9319 - val_mae: 1.3083
Epoch 4/50
855/855 [==============================] - 2s 2ms/step - loss: 3.6609 - mse: 3.6609 - mae: 1.4578 - val_loss: 2.7710 - val_mse: 2.7710 - val_mae: 1.2704
Epoch 5/50
855/855 [==============================] - 2s 2ms/step - loss: 3.5747 - mse: 3.5747 - mae: 1.4392 - val_loss: 2.8815 - val_mse: 2.8815 - val_mae: 1.2991
Epoch 6/50
855/855 [==============================] - 2s 2ms/step - loss: 3.6079 - mse: 3.6079 - mae: 1.4514 - val_loss: 2.7496 - val_mse: 2.7496 - val_mae: 1.2711
Epoch 7/50
855

In [49]:
xnew=np.array([[913,1072,1595,3736,5340]])
ynew= model.predict(xnew)
print("X=%s, Predicted=%s" % (xnew[0], ynew[0]))

1/1 [==============================] - 0s 67ms/step
X=[ 913 1072 1595 3736 5340], Predicted=[2331.1582]


In [56]:
model.save('predtest.h5')

In [62]:
import tensorflow as tf

h5_model_path = 'predtest.h5'

# Load the Keras model
model = tf.keras.models.load_model(h5_model_path)

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpb8k8rydy\assets


INFO:tensorflow:Assets written to: C:\Users\Admin\AppData\Local\Temp\tmpb8k8rydy\assets


In [63]:
xnew=np.array([[913,1072,1595,3736,5340]])
ynew= model.predict(xnew)
print("X=%s, Predicted=%s" % (xnew[0], ynew[0]))

1/1 [==============================] - 0s 114ms/step
X=[ 913 1072 1595 3736 5340], Predicted=[2331.1582]


In [85]:
# Select random samples to predict the voltage
import random

# Select a random starting point
random_start = random.randint(0, len(df)-seq_len)
print(random_start)

# Create a random sample of 5 time steps
xnew = df['Voltage'][random_start:random_start+seq_len].values
xnew = xnew.reshape((1, seq_len))

# Predict the voltage
ynew = model.predict(xnew)
print(f"X={xnew}")
print(f"Predicted={ynew}")
# Actual voltage
print(f"Actual={(round(df['Voltage'][random_start+seq_len],5))}")

33506
1/1 [==============================] - 0s 21ms/step
X=[[245.64566667 242.10933333 243.06133333 239.85716667 243.37783333]]
Predicted=[[243.45512]]
Actual=242.17833


In [ ]:
# print(history.history.keys())
# # "Loss"
# plt.plot(history.history['loss'])
# plt.plot(history.history['mse'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()